In [16]:
import os, sys, re, json, time, unittest
import itertools, collections
from importlib import reload

import numpy as np
from scipy import stats

import nltk
from collections import defaultdict

# Helper libraries for this notebook
from common import utils, vocabulary


In [8]:
from trigram_addk import next_proba

In [11]:
import nltk
nltk.download('reuters')
corpus = nltk.corpus.reuters
token_feed = (utils.canonicalize_word(w) for w in corpus.words())
vocab = vocabulary.Vocabulary(token_feed)
V=30000
print("Vocabulary size: {:,}".format(vocab.size))
print("Most common unigrams:")
for word, count in vocab.unigram_counts.most_common(10):
    print("\"{:s}\": {:,}".format(word, count))

train_sents, test_sents = utils.get_train_test_sents(corpus, split=0.8, shuffle=False)
vocab = vocabulary.Vocabulary((utils.canonicalize_word(w) for w in utils.flatten(train_sents)), size=V)
print("Train set vocabulary: {:,} words".format(vocab.size))


[nltk_data] Downloading package reuters to
[nltk_data]     /Users/nathanielschub/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
Vocabulary size: 29,598
Most common unigrams:
".": 94,687
",": 72,360
"the": 69,277
"DGDG": 48,960
"DG": 46,478
"DGDGDG": 40,589
"of": 36,779
"to": 36,400
"in": 29,253
"and": 25,648
Loaded 54,716 sentences (1.72092e+06 tokens)
Training set: 43,772 sentences (1,383,234 tokens)
Test set: 10,944 sentences (337,683 tokens)
Train set vocabulary: 26,822 words


In [13]:
from nltk.corpus import reuters
a= reuters.raw().lower()
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
#import re
a = word_tokenize(a)
#tokenizer = RegexpTokenizer(r'\w+')
#tokenizer.tokenize(a)
#a = a.split()
print(len(a))

1544239


In [14]:
print(a[:100])

['asian', 'exporters', 'fear', 'damage', 'from', 'u.s.-japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'u.s.', 'and', 'japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asia', "'s", 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far-reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.', 'they', 'told', 'reuter', 'correspondents', 'in', 'asian', 'capitals', 'a', 'u.s.', 'move', 'against', 'japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'u.s.', 'and', 'lead', 'to', 'curbs', 'on', 'american', 'imports', 'of', 'their', 'products', '.', 'but', 'some', 'exporters', 'said', 'that', 'while', 'the', 'conflict', 'would', 'hurt', 'them', 'in', 'the', 'long-run', ',', 'in', 'the', 'short-term', 'tokyo', "'s", 'loss', 'might', 'be', 'their', 'gain', '.', 'the', 'u.s.', 'has', 'said', 'it']


In [17]:
words = a
wordset = set()

counts = defaultdict(lambda: defaultdict(lambda: 0.0))
context_totals = dict()
w_1, w_2 = None, None
for word in words:
    wordset.add(word)
    if w_1 is not None and w_2 is not None:
        counts[(w_2,w_1)][word] += 1
            # Update context
    w_2 = w_1
    w_1 = word
for key, value in counts.items():
    context_totals[key] = sum(value.values())
    
words = list(wordset)
V = len(words)
k = 3

In [18]:
print(counts[('the', 'u.s')])

defaultdict(<function <lambda>.<locals>.<lambda> at 0x1a18a2a620>, {'where': 1.0, ',': 5.0, 'and': 1.0, 'agriculture': 1.0, 'in': 2.0, 'to': 1.0, 'economy': 1.0, 'responded': 2.0, 'later': 1.0, '.': 2.0})


In [89]:
def next_proba(corpus_tokenized, x, context, k):
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    context_totals = dict()
    w_1, w_2 = None, None
    wordset = set()
    for word in corpus_tokenized:
        wordset.add(word)
        if w_1 is not None and w_2 is not None:
            counts[(w_2,w_1)][word] += 1
            # Update context
        w_2 = w_1
        w_1 = word
    for key, value in counts.items():
        context_totals[key] = sum(value.values())
    
    words = list(wordset)
    V = len(words)
    print(V)    
    
    
    
    
    C_abc = 0.0
    if counts.get(context, 0.0) == 0.0:
        C_abc = 0.0
    else:
        C_abc = counts.get(context, 0.0).get(x, 0.0)
                
    C_ab = context_totals.get(context, 0.0)
        
    return (C_abc +k)/(C_ab +(k*V))



In [87]:
next_proba(a,'economy',('the','u.s'), 5)

52916


2.2675994058889557e-05

In [81]:
b = ['my', 'name', 'is', 'nat', 'test', 'test', 'test']

next_proba(b, 'test', ('test','test'), 2)

5


0.2727272727272727

In [109]:
def eval_candidates(corpus_tokenized, list_target_words, context, k):
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    context_totals = dict()
    w_1, w_2 = None, None
    wordset = set()
    for word in corpus_tokenized:
        wordset.add(word)
        if w_1 is not None and w_2 is not None:
            counts[(w_2,w_1)][word] += 1
            # Update context
        w_2 = w_1
        w_1 = word
    for key, value in counts.items():
        context_totals[key] = sum(value.values())
    
    words = list(wordset)
    V = len(words)
    
    values=[]
    
    for word_x in list_target_words:
    
        C_abc = 0.0
        if counts.get(context, 0.0) == 0.0:
            C_abc = 0.0
        else:
            C_abc = counts.get(context, 0.0).get(word_x, 0.0)

        C_ab = context_totals.get(context, 0.0)

        values.append((word_x,(C_abc +k)/(C_ab +(k*V)) )) 
    return values




In [110]:
eval_candidates(b, ['test','tst'],('test','test'),2)

[('test', 0.2727272727272727), ('tst', 0.18181818181818182)]

In [111]:
eval_candidates(a, ['where',',','agriculture', 'economy'], ('the','u.s'), 2)

[('where', 2.8342261145594194e-05),
 (',', 6.613194267305312e-05),
 ('agriculture', 2.8342261145594194e-05),
 ('economy', 2.8342261145594194e-05)]